# Import des modules nécessaires : 

In [1]:
import pandas as pd
import numpy as np
import functions as fc
import imlconnect
from datetime import datetime, timedelta
import time

In [2]:
print("Entrez la date du jour svp ?\n")
j = input("Jour ? (format numérique svp!)\n")
while len(j)!=2:
    j = input("Jour ? (2 caractères et format numérique svp!)\n")
while True:
    try:
        val = int(str(j))
        while val<1 or val>31:
            j = input("Jour ? Entre 01 et 31 svp!)\n") 
            val = int(str(j))
    except ValueError:
        # Not a valid number
        print("Ce n'est pas un nombre entier\n")
        j = input("Jour ? (format numérique svp!)\n")
    else:
        print("Succès !\n")
        # No error; stop the loop
        break  

m = input("Mois ? (format numérique svp!)\n")
while len(m)!=2:
    m = input("Mois ? (2 caractères et format numérique svp!)\n")
while True:
    try:
        val1 = int(str(m))
        while val1<1 or val1>12:
            m = input("Mois ? Entre 01 et 12 svp!)\n") 
            val1 = int(str(m))
    except ValueError:
        # Not a valid number
        print("Ce n'est pas un nombre entier\n")
        m = input("Mois ? (format numérique svp!)\n")
    else:
        print("Succès !\n")# No error; stop the loop
        break

a = input("Année en cours? (format numérique svp!)\n")
while len(a)!=4:
    a = input("Année ? (4 caractères et format numérique svp!)\n")
while True:
    try:
        val2 = int(str(a))
        while val2!=datetime.now().year:
            a = input("Année doit être année en cours svp!)\n") 
            val2 = int(str(a))
    except ValueError:
        # Not a valid number
        print("Ce n'est pas un nombre entier\n")
        a = input("Année ? (format numérique svp!)\n")
    else:
        print("Succès !\n")# No error; stop the loop
        break

Entrez la date du jour svp ?



Jour ? (format numérique svp!)
 29


Succès !



Mois ? (format numérique svp!)
 12


Succès !



Année en cours? (format numérique svp!)
 2022


Succès !



In [3]:
date_saisie = pd.to_datetime(str(val2)+"-"+str(val1)+"-"+str(val)).strftime("%Y-%m-%d")
print("On va récupérer les données dans LinkyPilot en fonction de l'index date (J-1), vous devez retirer 1 jour à la date saisie.\n")
Q = input("Voulez retirer des jours à la date d'analyse ? oui ou non ?\n")
while Q!='oui' and Q!='non':
    print("Ecrivez correctement oui ou non en minuscule svp!\n")
    Q = input("Voulez retirer des jours à la date d'analyse ? oui ou non ?\n")
if Q=="oui":
    jour_à_retirer = input("Entrer le nombre de jour à retirer svp ?\n")
    while True:
        try:
            jour_à_retirer = int(str(jour_à_retirer))
        except ValueError:
            print("Ce n'est pas un nombre entier\n")
            jour_à_retirer = input("Entrez un nombre entier svp ?\n")
        else:
            print("Succès !\n")
            break
    date_analyse = pd.to_datetime(date_saisie) - timedelta(days=jour_à_retirer)
    date_analyse = date_analyse.strftime("%Y-%m-%d")
             
elif Q=='non':
    date_analyse = pd.to_datetime(date_saisie) - timedelta(days=0)
    date_analyse = date_analyse.strftime("%Y-%m-%d")
print("La date d'analyse est : \n",date_analyse) 
print("La date saisie est : \n",date_saisie)

On va récupérer les données dans LinkyPilot en fonction de l'index date (J-1), vous devez retirer 1 jour à la date saisie.



Voulez retirer des jours à la date d'analyse ? oui ou non ?
 oui
Entrer le nombre de jour à retirer svp ?
 1


Succès !

La date d'analyse est : 
 2022-12-28
La date saisie est : 
 2022-12-29


In [4]:
start = time.process_time()

In [5]:
my_conn_sup = imlconnect.Sql("lsup_conn", 
                  key_path='/var/projects/iml/rstudio/D86553/python_conn/imlconnect.key', 
                  config_file_path='/var/projects/iml/rstudio/D86553/python_conn/config_imlconnect.yml',
                  open_connection=True)
print("fin de traitement")

fin de traitement


In [6]:
# Process_time affiche la partie seconde de l'heure donc ici on fait différence heure start et stop pour connaitre la durée de la requête


df_da_prm_usine = my_conn_sup.query("""
    select LOC_CODE DR,PDC_ID , PDC_ID_C,  PDC_ID_PRM_DEC ID_PRM, PDC_PDK_COM_FK PDK_COM, DA.DAN_NUMERO_DOSSIER NUMERO_DA, statut_da.sda_libelle STATUT_DA , PDC_DATE_DECOUVERTE DATE_DECOUVERTE
                                           from lspmet.t_dossier_analyse DA
                                           join lspmet.T_ref_statut_dossier_analyse statut_da on DA.DAN_STATUT_DOSSIER_ANALYSE_FK = statut_da.sda_id
                                           join lspmet.t_dysfonctionnement DYS on DYS.DYS_ID= DA.DAN_DYSFONCTIONNEMENT_FK
                                           JOIN lspmet.T_REF_TYPE_DYSFONCTIONNEMENT TDY ON DYS.DYS_TYPE_DYSFONCTIONNEMENT_FK = TDY.TDY_ID
                                           JOIN lspmet.t_pdc on dys_id_objet = pdc_id
                                           join lspmet.t_localisation_dossier lod on lod.lod_dossier_fk = da.dan_id
                                           join lspmet.t_ref_localisation_erdf loc on loc.loc_id=lod.lod_localisation_fk
                                           WHERE TDY_ID = 34 and sda_id=2 and PDC_ID_PRM_DEC='00000000000000'
    """)



## Méthode pour accéder à une colonne d'un dataframe: 
### nom_df["nom_colonne"]
### nom_df['nom_colonne']
### nom_df.nom_colonne, elle ne fonctionne que si nom_colonne ne contient pas d'espaces
## Pour accéder à une ligne, on utilise la méthode .iloc de la librairie pandas (voir pandas)


In [7]:
# On appelle la fonction filtre_date pour avoir notre df qui contient les lignes voulus
df_filtré = fc.filtre_date(df_da_prm_usine,"date_decouverte",date_saisie,10)
df_filtré = df_filtré.sort_values(by="date_decouverte",ascending = False).reset_index(drop = True)


In [8]:
sql_ini = "select ID_PDC, ID_PRM PRM_C_ARRET_COLLECTE, ID_PDK_COM PDK_COM, ID_COMPTEUR, NB_JOUR_NON_COLLECTE from lptdwh_dmt.fait_pdc fpdc inner join filter_list f on f.val = fpdc.ID_PDK_COM where nb_jour_non_collecte > 7 and nb_c_reconcilie = 1 and id_jour = to_date('"+ date_analyse + "','yyyy-mm-dd')"
#Fonction de creation complete du sql avec df = dataframe, value = Colonne du tableau utilisé pour filtre, sql_init = sql à executer  
sql = fc.create_sql_with_df(df_filtré,'pdk_com',sql_ini)


In [9]:
my_conn_pilot = imlconnect.Sql("pilot_conn", 
                      key_path='/var/projects/iml/rstudio/D86553/python_conn/imlconnect.key', 
                      config_file_path='/var/projects/iml/rstudio/D86553/python_conn/config_imlconnect.yml',
                      open_connection=True)

In [10]:
df_INFO_Collecte = my_conn_pilot.query(sql)

In [11]:

df_INFO_Collecte.nb_jour_non_collecte = pd.to_timedelta(df_INFO_Collecte.nb_jour_non_collecte,unit='days',errors='coerce')

In [12]:
# date_analyse = date_saisie - le nombre de jour retranché (en input)
df_INFO_Collecte_Analyse = df_INFO_Collecte.assign(date_arret_collecte = pd.to_datetime(date_analyse) - df_INFO_Collecte.nb_jour_non_collecte)


In [13]:
# On vérifie les compteurs en arrêt de collecte s'ils sont perdus dans Linkycoeur
df_Analyse = pd.merge(df_filtré,df_INFO_Collecte_Analyse,on ='pdk_com',how = 'inner' )
  # Pour rappel date de pose = date du compte-rendu de pose du technicien poseur
  # Date de découverte = date à laquelle le compteur a communiqué pour la première fois avec le concentrateur et le SI pour s'authentifier
  # Réconciliation nécessite le CR de pose accepté par le SI (LinkyParc) et la réussite de découverte 

In [14]:
  # chercher des correspondances entre DA PRM Usine et Infos de collecte FAIT_PDC
df_Analyse = df_Analyse.assign(delta_1 =  df_Analyse.date_decouverte - df_Analyse.date_arret_collecte)

In [15]:
# delta_1 = différence en unité Jours entre DATE DECOUVERTE ET DATE ARRET COLLECTE

df_Analyse.delta_1 = df_Analyse.delta_1.dt.components['days']

In [16]:
# je veux que la différence soit d'un jour ou moins
df_Analyse_delta = df_Analyse.query("delta_1 <=1 and delta_1 >=0")


In [17]:
df_Analyse_delta = df_Analyse_delta.assign(nb_correspondance = 1)

In [18]:
df_Analyse_delta = df_Analyse_delta.drop('nb_correspondance',axis=1).merge(df_Analyse_delta.drop('delta_1',axis=1).groupby(by= ["pdc_id"],as_index = False).nb_correspondance.sum(), on ='pdc_id', how='outer')

In [19]:
df_Analyse_delta = df_Analyse_delta.assign(nb_presence_c_arret = 1)
df_Analyse_delta = df_Analyse_delta.drop('nb_presence_c_arret',axis=1).merge(df_Analyse_delta.drop(['delta_1','nb_correspondance'],axis=1).groupby(by= "id_compteur",as_index = False).nb_presence_c_arret.sum(), on ='id_compteur', how='outer')

In [20]:
df_Analyse_delta

dr          pdc_id      pdc_id_c          id_prm           pdk_com  \
0       DRGUA  T_042164321947  042164321947  00000000000000  97116R0065-15460   
1       DRBOU  T_811775264785  811775264785  00000000000000  58171T0015-15179   
2     DRCAZUR  T_042262017029  042262017029  00000000000000  83137R0358-03915   
3       DRAFC  T_032275247157  032275247157  00000000000000  39441S0029-63774   
4      DRIDFE  T_022275142458  022275142458  00000000000000  93071R0003-86621   
...       ...             ...           ...             ...               ...   
1829  DRSIRHO  T_041763042028  041763042028  00000000000000  69034S0766-00855   
1830  DRSIRHO  T_841775283425  841775283425  00000000000000  69256R0120-10339   
1831   DRIDFE  T_041763029918  041763029918  00000000000000  93005R0504-03487   
1832   DRIDFE  T_031662686005  031662686005  00000000000000  93071R0014-04548   
1833   DRIDFE  T_041663153502  041663153502  00000000000000  77284T0094-02968   

           numero_da statut_da         date_decouverte          id_pdc  \
0     DA221222FMWJFO  En cours 2022-12-18 16:46:00.104  PDC00015421002   
1     DA221222FMWHAF  En cours 2022-12-18 15:27:23.667  PDC00023006524   
2     DA221222FMWAGS  En cours 2022-12-18 11:18:48.436  PDC00001803690   
3     DA221222FMVWOA  En cours 2022-12-18 01:55:44.788  PDC00036173315   
4     DA221221FMVVXA  En cours 2022-12-17 23:26:56.322  PDC00035971544   
...              ...       ...                     ...             ...   
1829  DA200812EDAIAS  En cours 2018-05-22 11:38:56.099  PDC00000093615   
1830  DA200812EDAJQZ  En cours 2017-11-23 15:17:13.970  PDC00005277223   
1831  DA200812EDAJBN  En cours 2017-11-07 13:51:56.146  PDC00001884448   
1832  DA200812EDAIGV  En cours 2017-10-05 08:05:54.477  PDC00002894774   
1833  DA200812EDAHKE  En cours 2017-06-23 09:55:09.221  PDC00003464396   

     prm_c_arret_collecte   id_compteur nb_jour_non_collecte  \
0          76108947310126  841775135928              10 days   
1          12532272050675  041864984970              10 days   
2          25350940638150  031662628211              10 days   
3          06523154835332  022275053212              10 days   
4          22557452885575  022164401696              11 days   
...                   ...           ...                  ...   
1829       19180607774524  031067015660            1681 days   
1830       19152821888445  041764130594            1861 days   
1831       22595658381589  031662658795            1877 days   
1832       22541968113132  021662947795            1911 days   
1833       22186685943305  041662424049            2014 days   

     date_arret_collecte  delta_1  nb_correspondance  nb_presence_c_arret  
0             2022-12-18        0                  1                    1  
1             2022-12-18        0                  1                    1  
2             2022-12-18        0                  1                    1  
3             2022-12-18        0                  1                    1  
4             2022-12-17        0                  1                    1  
...                  ...      ...                ...                  ...  
1829          2018-05-22        0                  1                    1  
1830          2017-11-23        0                  1                    1  
1831          2017-11-07        0                  1                    1  
1832          2017-10-04        1                  1                    1  
1833          2017-06-23        0                  1                    1  

[1834 rows x 16 columns]

In [21]:
df_Analyse_delta = df_Analyse_delta.query("nb_correspondance ==1 and nb_presence_c_arret==1")
df_Analyse_delta

dr          pdc_id      pdc_id_c          id_prm           pdk_com  \
0       DRGUA  T_042164321947  042164321947  00000000000000  97116R0065-15460   
1       DRBOU  T_811775264785  811775264785  00000000000000  58171T0015-15179   
2     DRCAZUR  T_042262017029  042262017029  00000000000000  83137R0358-03915   
3       DRAFC  T_032275247157  032275247157  00000000000000  39441S0029-63774   
4      DRIDFE  T_022275142458  022275142458  00000000000000  93071R0003-86621   
...       ...             ...           ...             ...               ...   
1829  DRSIRHO  T_041763042028  041763042028  00000000000000  69034S0766-00855   
1830  DRSIRHO  T_841775283425  841775283425  00000000000000  69256R0120-10339   
1831   DRIDFE  T_041763029918  041763029918  00000000000000  93005R0504-03487   
1832   DRIDFE  T_031662686005  031662686005  00000000000000  93071R0014-04548   
1833   DRIDFE  T_041663153502  041663153502  00000000000000  77284T0094-02968   

           numero_da statut_da         date_decouverte          id_pdc  \
0     DA221222FMWJFO  En cours 2022-12-18 16:46:00.104  PDC00015421002   
1     DA221222FMWHAF  En cours 2022-12-18 15:27:23.667  PDC00023006524   
2     DA221222FMWAGS  En cours 2022-12-18 11:18:48.436  PDC00001803690   
3     DA221222FMVWOA  En cours 2022-12-18 01:55:44.788  PDC00036173315   
4     DA221221FMVVXA  En cours 2022-12-17 23:26:56.322  PDC00035971544   
...              ...       ...                     ...             ...   
1829  DA200812EDAIAS  En cours 2018-05-22 11:38:56.099  PDC00000093615   
1830  DA200812EDAJQZ  En cours 2017-11-23 15:17:13.970  PDC00005277223   
1831  DA200812EDAJBN  En cours 2017-11-07 13:51:56.146  PDC00001884448   
1832  DA200812EDAIGV  En cours 2017-10-05 08:05:54.477  PDC00002894774   
1833  DA200812EDAHKE  En cours 2017-06-23 09:55:09.221  PDC00003464396   

     prm_c_arret_collecte   id_compteur nb_jour_non_collecte  \
0          76108947310126  841775135928              10 days   
1          12532272050675  041864984970              10 days   
2          25350940638150  031662628211              10 days   
3          06523154835332  022275053212              10 days   
4          22557452885575  022164401696              11 days   
...                   ...           ...                  ...   
1829       19180607774524  031067015660            1681 days   
1830       19152821888445  041764130594            1861 days   
1831       22595658381589  031662658795            1877 days   
1832       22541968113132  021662947795            1911 days   
1833       22186685943305  041662424049            2014 days   

     date_arret_collecte  delta_1  nb_correspondance  nb_presence_c_arret  
0             2022-12-18        0                  1                    1  
1             2022-12-18        0                  1                    1  
2             2022-12-18        0                  1                    1  
3             2022-12-18        0                  1                    1  
4             2022-12-17        0                  1                    1  
...                  ...      ...                ...                  ...  
1829          2018-05-22        0                  1                    1  
1830          2017-11-23        0                  1                    1  
1831          2017-11-07        0                  1                    1  
1832          2017-10-04        1                  1                    1  
1833          2017-06-23        0                  1                    1  

[1063 rows x 16 columns]

In [22]:
# Requête LinkyCoeur sur table CK_LIEN_COURANT (RCM = référentiel comptage, RDS = demandes de service, RDM = données mesurées)
# Requête sur ETAT_LIEN_C_K =1 (1 = Perdu, autre valeur = non perdu)
# Vérifier que la perte est l'état courant du C
# Je veux que mes compteurs en arrêt de collecte soient au statut PERDU = sur-vérification vis à vis de l'arrêt de collecte
my_conn_rcm = imlconnect.Sql("rcm_conn", 
                      key_path='/var/projects/iml/rstudio/D86553/python_conn/imlconnect.key', 
                      config_file_path='/var/projects/iml/rstudio/D86553/python_conn/config_imlconnect.yml',
                      open_connection=True)
        

In [23]:
sql_ini1 = "select DATE_DEBUT DATE_PERTE, LPAD(ID_C,12,'0') ID_COMPTEUR from rcm.ck_lien_courant l inner join filter_list f on f.val = l.ID_C where ETAT_LIEN_C_K=1"
sql1 = fc.create_sql_with_df(df_Analyse_delta,'id_compteur',sql_ini1)

In [24]:

df_Perte_ArretColl= my_conn_rcm.query(sql1)


In [25]:
df_Perte_ArretColl = df_Perte_ArretColl.assign(date_dern_com = df_Perte_ArretColl.date_perte - timedelta(seconds = 86400))


In [26]:
sql_ini2 = "select DATE_DEBUT DATE_DERN_PIL02, LPAD(ID_C,12,'0') ID_COMPTEUR from rcm.ck_lien_courant l inner join filter_list f on f.val = l.ID_C where ETAT_LIEN_C_K in (2,3)"
sql2 = fc.create_sql_with_df(df_Analyse_delta,'id_compteur',sql_ini2)

In [27]:
df_EtatCourant_CK_PIL02 = my_conn_rcm.query(sql2)

In [28]:
sql_ini3 = "select e.compteur_id serialnum, s.sous_type emplacement,type_parc type, sous_type_parc sous_type from lptdwh_ods.equipement e inner join filter_list f on f.val = e.compteur_id inner join lptdwh_ods.logistique l on e.equipement_id=l.equipement_id inner join lptdwh_ods.site s on l.site_id_arrivee = s.site_id where t_dernier_mouvement=1"
sql3 = fc.create_sql_with_df(df_Analyse_delta,'pdc_id_c',sql_ini3)

In [29]:
df_Infos_Emplacement = my_conn_pilot.query(sql3)

In [30]:
df_Analyse_Finale = pd.merge(df_Analyse_delta,df_Perte_ArretColl,on="id_compteur",how = "left")
df_Analyse_Finale = df_Analyse_Finale.assign(delta_2 = df_Analyse_Finale.date_decouverte - df_Analyse_Finale.date_dern_com)
  # Calcul de la différence entre date découverte du Compteur 000000 et date de dernière communication du compteur en arrêt de collecte correspondant
  # Différence entre deux dates ymd_hms = en minutes

In [31]:
df_Analyse_Finale["delta_2"] = df_Analyse_Finale["delta_2"].dt.total_seconds()

In [32]:
df_Analyse_Finale["delta_2"].isnull().sum()

24

In [33]:
# résultat en secondes 
df_Analyse_Finale.insert(0,"probabilite_forte",df_Analyse_Finale["delta_2"].apply(lambda x: False if abs(x) > 10800 else True))
# probabilité forte si moins de 3H d'écart entre la dernière comm estimée et la découverte 

In [34]:
df_Finale = pd.merge(df_Analyse_Finale,df_EtatCourant_CK_PIL02,on='id_compteur',how ='left')
  # Vérifier que Compteur en arrêt de collecte n'a pas tenté de se redécouvrir ultérieurement à la date de découverte du C Découvert PRM USINE

In [35]:
df_Finale= df_Finale.assign(delta_3 = np.where(df_Finale.date_dern_pil02>df_Finale.date_decouverte, "KO", np.NaN))

In [36]:
df_Finale

probabilite_forte       dr          pdc_id      pdc_id_c  \
0                  True    DRGUA  T_042164321947  042164321947   
1                  True    DRBOU  T_811775264785  811775264785   
2                  True  DRCAZUR  T_042262017029  042262017029   
3                  True    DRAFC  T_032275247157  032275247157   
4                 False   DRIDFE  T_022275142458  022275142458   
...                 ...      ...             ...           ...   
1058               True  DRSIRHO  T_041763042028  041763042028   
1059              False  DRSIRHO  T_841775283425  841775283425   
1060               True   DRIDFE  T_041763029918  041763029918   
1061              False   DRIDFE  T_031662686005  031662686005   
1062               True   DRIDFE  T_041663153502  041663153502   

              id_prm           pdk_com       numero_da statut_da  \
0     00000000000000  97116R0065-15460  DA221222FMWJFO  En cours   
1     00000000000000  58171T0015-15179  DA221222FMWHAF  En cours   
2     00000000000000  83137R0358-03915  DA221222FMWAGS  En cours   
3     00000000000000  39441S0029-63774  DA221222FMVWOA  En cours   
4     00000000000000  93071R0003-86621  DA221221FMVVXA  En cours   
...              ...               ...             ...       ...   
1058  00000000000000  69034S0766-00855  DA200812EDAIAS  En cours   
1059  00000000000000  69256R0120-10339  DA200812EDAJQZ  En cours   
1060  00000000000000  93005R0504-03487  DA200812EDAJBN  En cours   
1061  00000000000000  93071R0014-04548  DA200812EDAIGV  En cours   
1062  00000000000000  77284T0094-02968  DA200812EDAHKE  En cours   

             date_decouverte          id_pdc  ... nb_jour_non_collecte  \
0    2022-12-18 16:46:00.104  PDC00015421002  ...              10 days   
1    2022-12-18 15:27:23.667  PDC00023006524  ...              10 days   
2    2022-12-18 11:18:48.436  PDC00001803690  ...              10 days   
3    2022-12-18 01:55:44.788  PDC00036173315  ...              10 days   
4    2022-12-17 23:26:56.322  PDC00035971544  ...              11 days   
...                      ...             ...  ...                  ...   
1058 2018-05-22 11:38:56.099  PDC00000093615  ...            1681 days   
1059 2017-11-23 15:17:13.970  PDC00005277223  ...            1861 days   
1060 2017-11-07 13:51:56.146  PDC00001884448  ...            1877 days   
1061 2017-10-05 08:05:54.477  PDC00002894774  ...            1911 days   
1062 2017-06-23 09:55:09.221  PDC00003464396  ...            2014 days   

     date_arret_collecte delta_1 nb_correspondance  nb_presence_c_arret  \
0             2022-12-18       0                 1                    1   
1             2022-12-18       0                 1                    1   
2             2022-12-18       0                 1                    1   
3             2022-12-18       0                 1                    1   
4             2022-12-17       0                 1                    1   
...                  ...     ...               ...                  ...   
1058          2018-05-22       0                 1                    1   
1059          2017-11-23       0                 1                    1   
1060          2017-11-07       0                 1                    1   
1061          2017-10-04       1                 1                    1   
1062          2017-06-23       0                 1                    1   

                  date_perte           date_dern_com    delta_2  \
0    2022-12-19 16:38:52.261 2022-12-18 16:38:52.261    427.843   
1    2022-12-19 14:25:58.304 2022-12-18 14:25:58.304   3685.363   
2    2022-12-19 13:31:34.092 2022-12-18 13:31:34.092  -7965.656   
3    2022-12-19 00:11:57.135 2022-12-18 00:11:57.135   6227.653   
4    2022-12-18 18:20:47.586 2022-12-17 18:20:47.586  18368.736   
...                      ...                     ...        ...   
1058 2018-05-23 10:36:53.785 2018-05-22 10:36:53.785   3722.314   
1059 2017-11-24 23:47:40.347 2017-11-23 23:47:40.347 -30626.377   
106

In [37]:
df = pd.merge(df_Finale[df_Finale.delta_3=="nan"], df_Infos_Emplacement.rename(columns={'serialnum':'pdc_id_c'}),on = 'pdc_id_c',how = 'left') # jointure avec état logistique pour récupérer l'emplacement de chaque compteur

In [38]:
  # Je consitue mon fichier final en sélectionnant les champs nécessaires +1 renommage
df_Fichier = df[['dr','numero_da','id_prm','pdc_id','pdc_id_c','pdk_com','date_decouverte','id_compteur','date_arret_collecte','prm_c_arret_collecte','emplacement','type','sous_type','id_compteur']]


In [39]:
df_Fichier = df_Fichier.rename(columns={'id_compteur':'c_arret_collecte'})

In [40]:
df_Fichier

dr       numero_da          id_prm          pdc_id      pdc_id_c  \
0       DRGUA  DA221222FMWJFO  00000000000000  T_042164321947  042164321947   
1       DRBOU  DA221222FMWHAF  00000000000000  T_811775264785  811775264785   
2     DRCAZUR  DA221222FMWAGS  00000000000000  T_042262017029  042262017029   
3       DRAFC  DA221222FMVWOA  00000000000000  T_032275247157  032275247157   
4      DRIDFE  DA221221FMVVXA  00000000000000  T_022275142458  022275142458   
...       ...             ...             ...             ...           ...   
1051  DRSIRHO  DA200812EDAIAS  00000000000000  T_041763042028  041763042028   
1052  DRSIRHO  DA200812EDAJQZ  00000000000000  T_841775283425  841775283425   
1053   DRIDFE  DA200812EDAJBN  00000000000000  T_041763029918  041763029918   
1054   DRIDFE  DA200812EDAIGV  00000000000000  T_031662686005  031662686005   
1055   DRIDFE  DA200812EDAHKE  00000000000000  T_041663153502  041663153502   

               pdk_com         date_decouverte c_arret_collecte  \
0     97116R0065-15460 2022-12-18 16:46:00.104     841775135928   
1     58171T0015-15179 2022-12-18 15:27:23.667     041864984970   
2     83137R0358-03915 2022-12-18 11:18:48.436     031662628211   
3     39441S0029-63774 2022-12-18 01:55:44.788     022275053212   
4     93071R0003-86621 2022-12-17 23:26:56.322     022164401696   
...                ...                     ...              ...   
1051  69034S0766-00855 2018-05-22 11:38:56.099     031067015660   
1052  69256R0120-10339 2017-11-23 15:17:13.970     041764130594   
1053  93005R0504-03487 2017-11-07 13:51:56.146     031662658795   
1054  93071R0014-04548 2017-10-05 08:05:54.477     021662947795   
1055  77284T0094-02968 2017-06-23 09:55:09.221     041662424049   

     date_arret_collecte prm_c_arret_collecte  \
0             2022-12-18       76108947310126   
1             2022-12-18       12532272050675   
2             2022-12-18       25350940638150   
3             2022-12-18       06523154835332   
4             2022-12-17       22557452885575   
...                  ...                  ...   
1051          2018-05-22       19180607774524   
1052          2017-11-23       19152821888445   
1053          2017-11-07       22595658381589   
1054          2017-10-04       22541968113132   
1055          2017-06-23       22186685943305   

                              emplacement     type     sous_type  \
0                              Landis Gyr  MAGASIN  Constructeur   
1                LINKY ENEDIS DECIZE (VE)  MAGASIN       Guichet   
2               LINKY SITE TOULON (Mardi)  MAGASIN       Guichet   
3                         ENEDIS LINKY MA  MAGASIN       Guichet   
4       LINKY ERDF-GRDF La Courneuve (LU)  MAGASIN       Guichet   
...                                   ...      ...           ...   
1051                PICARD - LYON - Lundi  MAGASIN   Prestataire   
1052                SOBECA CORBAS - Lundi  MAGASIN   Prestataire   
1053                TERCA Lagny (MAR/JEU)  MAGASIN   Prestataire   
1054  LINKY ERDF Exploit  Noisy le sec (L  MAGASIN       Guichet   
1055                  Ent. SN DUVAL (jeu)  MAGASIN   Prestataire   

     c_arret_collecte  
0        841775135928  
1        041864984970  
2        031662628211  
3        022275053212  
4        022164401696  
...               ...  
1051     031067015660  
1052     041764130594  
1053     031662658795  
1054     021662947795  
1055     041662424049  

[1056 rows x 14 columns]

In [41]:
#On exporte df_Fichier en fichier csv de moins de 1000 lignes dans le répertoire où on se trouve dans le dossier "export" à créer au préalable dans l'espace jupyter et à vider si besoin pour éviter de créer plein de dossier

import os 
directory = os.getcwd()
rep=directory+"/export_pilot_rapport_da/"

fc.split_df_to_csv(df_Fichier,999,rep,date_saisie,"DA_absence_crpose_prm_usine")

# zip du dossier
import shutil as sh
sh.make_archive('export_pilot_rapport_da','zip',rep)

'/var/projects/iml/rstudio/QN0BEEEL/my_folder/export_pilot_rapport_da.zip'